In [189]:
#Import librairies
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

#Get wikipedia data
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,"lxml")
My_table = soup.find("table",{"class":"wikitable sortable"})


In [190]:
#Create DataFrame from 3 first columns
PostalCode = []
Borough = []
Neighborhood = []
df = pd.DataFrame()

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 1:
        A = cells[0].find(text=True)
        B = cells[1].find(text=True)
        C = cells[2].find(text=True)
        PostalCode.append(A)
        Borough.append(B)
        Neighborhood.append(C)
        
df['PostalCode'] = PostalCode
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [191]:
#Retrieve "Not assigned" Borough
df = df[df['Borough'] != "Not assigned"]

#Replace "Not assigned" Neighborhood
for i in range(len(df['Borough'])):
    if df['Neighborhood'].iloc[i] == "Not assigned":
        df['Neighborhood'].iloc[i] = df['Borough'] .iloc[i] 
        
        
df = df.sort_values(by=['PostalCode'])

m = len(df['PostalCode'])
df2 = df

#Save number of distinct value of Postal Code
n = df['PostalCode'].nunique()

In [192]:

a = df['PostalCode'].iloc[0]
b = df['Borough'].iloc[0] 
c = df['Neighborhood'].iloc[0]

for i in range(m-1):
    i = i + 1
    if df['PostalCode'].iloc[i] != a:
        df2['PostalCode'].iloc[i]  = df['PostalCode'].iloc[i] 
        df2['Borough'].iloc[i]  = df['Borough'].iloc[i] 
        df2['Neighborhood'].iloc[i]  = ', '.join(set(list(df[df['PostalCode'] == df['PostalCode'].iloc[i]]['Neighborhood'])))
        a = df['PostalCode'].iloc[i]
    else:
        a = df['PostalCode'].iloc[i]
        df2['PostalCode'].iloc[i]  = "Not assigned"
        df2['Borough'].iloc[i]  = "Not assigned"
        df2['Neighborhood'].iloc[i]  = "Not assigned"

In [193]:
#Clean duplicates
df2 = df2[df2['Borough'] != "Not assigned"]

In [194]:
#Sanity check df2 size is equal to distinct value of Postal Code
n == len(df2['Borough'] )

True

In [197]:
#Sanity check on example "M5A"
df2[df2['PostalCode'] == "M5A"]

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [195]:
#Check the shape of the DataFrame
df2.shape

(103, 3)

In [212]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')


PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [227]:
df_final =df2.merge(geo, right_on = 'Postal Code' , left_on = 'PostalCode' ,how='left')

In [228]:
df_final1=df_final.drop(columns = ['PostalCode_x'])
df_final=df_final1.drop(columns = ['PostalCode_y'])
df_final.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Scarborough,Rouge,M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood\n, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae\n,M1H,43.773136,-79.239476
